In [ ]:
pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 7.5 MB/s 
     |████████████████████████████████| 120 kB 45.0 MB/s 
     |████████████████████████████████| 1.6 MB 50.9 MB/s 
     |████████████████████████████████| 82 kB 600 kB/s 
     |████████████████████████████████| 235 kB 47.9 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 48 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 45.4 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=9a94fb9fbeef949f9c28383ee4147e20a8ab60287bd5abbee3774f8c4cbe1978
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


In [ ]:
import tensorflow as tf
from PIL import Image
import io
import os
from object_detection.utils import dataset_util
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

In [ ]:
def get_img_info(filename):
    filename = filename.split('/')[-1]
    
    with open('/content/drive/MyDrive/archive/Images/'+filename.replace('.jpg','.txt')) as f:
        image_annotations = f.readlines()
        f.close()
    for i in range(len(image_annotations)):
        image_annotations[i] = image_annotations[i].split(' ')
    image_annotations = np.array(image_annotations).astype(np.float)
    #image_annotations[:,1:] = np.array(image_annotations)[:,1:].astype(np.float)
    image_annotations[:,0] = np.array(image_annotations)[:,0].astype(np.int)
    return image_annotations
    

In [ ]:
def create_tf_example(example):
    filename = example['filename']
    print(filename)
    with tf.io.gfile.GFile(filename, 'rb') as fid:
        encoded_jpg = io.BytesIO()
        image = Image.open('/content/drive/MyDrive/archive/Images/'+filename)
        image= image.resize((640,640))
        image.save(encoded_jpg, format='JPEG')
        encoded_jpg = encoded_jpg.getvalue()
    width, height = image.size
    # TODO(user): Populate the following variables from your example.
    image_format = b'jpg'# or b'png'
    
    xmins = example['xmin'] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = example['xmax'] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = example['ymin'] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = example['ymax']# List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = ['IssueBank','ReceiverName','AcNo','Amt','ChqNo','DateIss','Sign'] # List of string class name of bounding box (1 per box)
    classes_text = [x.encode('utf-8') for x in classes_text]
    classes = example['classid'].astype(np.int32) # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename.encode()),
      'image/source_id': dataset_util.bytes_feature(filename.encode()),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
writer = tf.io.TFRecordWriter("/content/drive/MyDrive/archive/train.record")

with open('/content/drive/MyDrive/archive/train.txt') as f:
    files = f.readlines()
    f.close()
# TODO(user): Write code to read in your dataset to examples variable

for example in files:
    example = example.replace("\n","")
    annotations = get_img_info(example)
    example = example.split('/')[-1]
    ann_dict = {"filename":example,"xmin":annotations[:,1],"ymin":annotations[:,2],
                "xmax":annotations[:,1]+annotations[:,3],"ymax":annotations[:,2]+annotations[:,4],
                "classid":annotations[:,0]}
    tf_example = create_tf_example(ann_dict)
    writer.write(tf_example.SerializeToString())

writer.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

Cheque083654.jpg
Cheque083655.jpg
Cheque083656.jpg
Cheque083657.jpg
Cheque083658.jpg
Cheque083659.jpg
Cheque083660.jpg
Cheque100828.jpg
Cheque100829.jpg
Cheque100830.jpg
Cheque100831.jpg
Cheque100832.jpg
Cheque100833.jpg
Cheque100834.jpg
Cheque100835.jpg
Cheque120611.jpg
Cheque120612.jpg
Cheque120613.jpg
Cheque120614.jpg
Cheque120615.jpg
Cheque120616.jpg
Cheque120617.jpg
Cheque120618.jpg
Cheque120619.jpg
Cheque120620.jpg
Cheque309061.jpg
Cheque309062.jpg
Cheque309063.jpg
Cheque309066.jpg
Cheque309067.jpg
Cheque309068.jpg
Cheque309069.jpg
Cheque309070.jpg
Cheque309071.jpg
Cheque309072.jpg
Cheque309073.jpg
Cheque309074.jpg
Cheque309075.jpg
Cheque309076.jpg
Cheque309077.jpg
Cheque309078.jpg
Cheque309079.jpg
Cheque309080.jpg
Cheque309082.jpg
Cheque309083.jpg
Cheque309084.jpg
Cheque309086.jpg
Cheque309089.jpg
Cheque309090.jpg
Cheque309091.jpg
Cheque309092.jpg
Cheque309093.jpg
Cheque309094.jpg
Cheque309095.jpg
Cheque309096.jpg
Cheque309097.jpg
Cheque309098.jpg
Cheque309099.jpg
Cheque309100.j

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2022-09-16 16:21:43--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.2.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90453990 (86M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v1_fp 100%[===================>]  86.26M  85.6MB/s    in 1.0s    

2022-09-16 16:21:44 (85.6 MB/s) - ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [90453990/90453990]



In [ ]:
!tar -xf ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!ls ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint

checkpoint  ckpt-0.data-00000-of-00001	ckpt-0.index


In [ ]:
def read_label_map(label_map_path):

    item_id = None
    item_name = None
    items = {}
    
    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

print ([i for i in read_label_map("object-detection.pbtxt")])

['IssueBank', 'ReceiverName', 'AcNo', 'Amt', 'ChqNo', 'DateIss', 'Sign']
